In [ ]:
# |export
#|default_exp p01_makemore


In [ ]:
# |export
import matplotlib
import matplotlib.pyplot as plt
plt.ion()
import os
import time
import pathlib
import tqdm
import argparse
import torch
import torch.nn.functional as F
from torch import tensor
from matplotlib.pyplot import plot, imshow, tight_layout, xlabel, ylabel, title, subplot, subplot2grid, grid, text, legend, figure, gcf, xlim, ylim
from torch import linspace, randn, randint, tanh



In [ ]:
class Args():
    def __init__(self):
        self.verbose=True
args=Args()


In [ ]:
# |export
start_time=time.time()
debug=True
_code_git_version="2765409fac4cfde8033da38c45dfd6358b1bc86c"
_code_repository="https://github.com/plops/cl-py-generator/tree/master/example/97_makemore/source/"
_code_generation_time="14:23:07 of Friday, 2024-05-10 (GMT+1)"
start_time=time.time()
debug=True
def lprint(msg, args):
    if ( args.verbose ):
        print("{} {}".format(((time.time())-(start_time)), msg))


In [ ]:
# |export
parser=argparse.ArgumentParser()
parser.add_argument("-v", "--verbose", help="enable verbose output", action="store_true")
args=parser.parse_args()


multi layer perceptron https://www.youtube.com/watch?v=TCH_1BHY58I 
reference bengio 2003 neural probabilistic language model


In [ ]:
# |export
words=open("/home/martin/stage/cl-py-generator/example/97_makemore/source/names.txt", "r").read().splitlines()
words[:10]


In [ ]:
# |export
chars=sorted(list(set("".join(words))))
stoi={s:((i)+(1)) for i, s in enumerate(chars)}
stoi["."]=0
itos={i:s for s, i in stoi.items()}
print(itos)


In [ ]:
# |export
block_size=3
X=[]
Y=[]
for w in words:
    print(w)
    context=(([0])*(block_size))
    for ch in ((w)+(".")):
        ix=stoi[ch]
        X.append(context)
        Y.append(ix)
        context=((context[1:])+([ix]))
X=tensor(X)
Y=tensor(Y)


In [ ]:
C=torch.randn((27,2,))


In [ ]:
((F.one_hot(tensor(5), num_classes=27).float())@(C))


In [ ]:
emb=C[X]
W1=torch.randn((6,100,))
b1=torch.randn((100,))
W2=torch.randn((100,27,))
b2=torch.randn((27,))


In [ ]:
h=torch.tanh(((((emb.view(-1, 6))@(W1)))+(b1)))


In [ ]:
logits=((((h)@(W2)))+(b2))
counts=logits.exp()
prob=((counts)/(counts.sum(1, keepdims=True)))


In [ ]:
# negative log-likelihood loss
loss=((-1)*(prob[torch.arange(32),Y].log().mean()))


In [ ]:
# |export
g=torch.Generator().manual_seed(2147483647)
C=torch.randn((27,2,), generator=g)
W1=torch.randn((6,100,))
b1=torch.randn((100,))
W2=torch.randn((100,27,))
b2=torch.randn((27,))
parameters=[C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad=True
n_parameters=sum(p.nelement() for p in parameters)
lprint("n_parameters={}".format(n_parameters), args)


In [ ]:
# forward
emb=C[X]
h=torch.tanh(((((emb.view(-1, 6))@(W1)))+(b1)))
logits=((((h)@(W2)))+(b2))
loss=F.cross_entropy(logits, Y)
# backward pass
# set gradients to zero
for p in parameters:
    p.grad=None
loss.backward()
# update
for p in parameters:
    p.data += (((-0.10    ))*(p.grad))


In [ ]:
# learning loop
for _ in range(10):
    # forward
    emb=C[X]
    h=torch.tanh(((((emb.view(-1, 6))@(W1)))+(b1)))
    logits=((((h)@(W2)))+(b2))
    loss=F.cross_entropy(logits, Y)
    lprint("loss.item()={}".format(loss.item()), args)
    # backward pass
    for p in parameters:
        p.grad=None
    loss.backward()
    # update
    for p in parameters:
        p.data += (((-0.10    ))*(p.grad))


In [ ]:
# |export
# learn with minibatch
for _ in range(10_000):
    ix=torch.randint(0, X.shape[0], (32,))
    # forward
    emb=C[X[ix]]
    h=torch.tanh(((((emb.view(-1, 6))@(W1)))+(b1)))
    logits=((((h)@(W2)))+(b2))
    loss=F.cross_entropy(logits, Y[ix])
    lprint("loss.item()={}".format(loss.item()), args)
    # backward pass
    for p in parameters:
        p.grad=None
    loss.backward()
    # update
    for p in parameters:
        p.data += (((-0.10    ))*(p.grad))
# report loss on entire data set
emb=C[X]
h=torch.tanh(((((emb.view(-1, 6))@(W1)))+(b1)))
logits=((((h)@(W2)))+(b2))
loss_full=F.cross_entropy(logits, Y)
lprint("loss_full.item()={}".format(loss_full.item()), args)


In [ ]:
# find a good learning rate, start with a very small lr and increase exponentially
lre=linspace(-3, 0, 1000)
lre=((10)**(lre))
lri=[]
lossi=[]
for lr in tqdm.tqdm(lre):
    ix=torch.randint(0, X.shape[0], (32,))
    # forward
    emb=C[X[ix]]
    h=torch.tanh(((((emb.view(-1, 6))@(W1)))+(b1)))
    logits=((((h)@(W2)))+(b2))
    loss=F.cross_entropy(logits, Y[ix])
    # backward pass
    for p in parameters:
        p.grad=None
    loss.backward()
    # update
    for p in parameters:
        p.data += ((-1)*(lr)*(p.grad))
    # track stats
    lri.append(lr)
    lossi.append(loss.item())


In [ ]:
plot(lre, lossi, alpha=(0.40    ))
grid()
